In [2]:
##########----- Reused code -----##########
# Last update: Sept 13, 2020
# By: Huitian Diao
###########################################

library(stringr)

code_dir <- getwd()
tool_dir <- unlist(strsplit(code_dir,"/"))
tool_dir <- head(tool_dir, length(tool_dir)-2)
tool_dir <- paste(c(tool_dir, "tools"), collapse="/")
source(file.path(tool_dir, "function_GSEA.R"))
base_dir <- gsub("/codes_local", "", code_dir)
base_dir

[1] "/media/pipkin/ROCKET-PRO/CD8_DEV_SC/1_IL2RAKO"

## Resampled Louvain cluster GSEA

In [3]:
##########----- Parameters-----##########
analysis_name <- c("0", "all")
genome <- "mm"
#########################################
analysis_subdir <- paste(paste(analysis_name[1], analysis_name[2], sep="-"), "Scanpy_out", sep="_")
sum.dir <- file.path(base_dir, '1_Scanpy', analysis_subdir, "3_GSEA")
dir.create(sum.dir, showWarnings = FALSE, recursive=TRUE)
setwd(sum.dir)


###----- Select gene signature
gs.dir <- file.path('/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm')
gs.pattern <- paste("_", genome, "_sigs.csv", sep="")
gs.files <- list.files(gs.dir, pattern=gs.pattern, full.names=TRUE)
gs.files

[1] "/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/2020_mouse_CD8_mm_sigs.csv"    
[2] "/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/CD4_mm_sigs.csv"               
[3] "/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/CD8_mm_sigs.csv"               
[4] "/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/Exp391_CD8_mm_sigs.csv"        
[5] "/media/pipkin/ROCKET-PRO/T_cell_signature_Reference/X_GeneSignatures_mm/Wolski_curated_CD8_mm_sigs.csv"

In [4]:
###----- Il2 dependent analysis
gs.files <- c('/media/pipkin/ROCKET-PRO/CD8_DEV_SC/9_Figures/2_IL2RAKO_Nascent/IL2_dependent_mm_sigs.csv')

In [ ]:
z.use.keys <- c('louvain', 'cell_type', 'louvain_celltype', 'louvain_celltype_general_withNAV')  

#####---------- For each classification method
for (z.use.key in z.use.keys) {
    z.file.name <- paste(z.use.key, 'mean_scaled_expr_z.csv', sep="_")
    z.file <- file.path(base_dir, '1_Scanpy', analysis_subdir, "1_avg_expr", z.file.name)
    z.tb <- read_csv(z.file)
    
    #####---------- For each signature collection, calculate GSEA & summarize
    for (gs.file in gs.files) {
        # Create sub directory for signature group
        gs.file.base <- gsub(gs.pattern, "",basename(gs.file))

        sep.dir <- paste(z.use.key, "sep", sep="_")
        wk.dir <- file.path(sum.dir, sep.dir, gs.file.base)
        dir.create(wk.dir, showWarnings = FALSE, recursive=TRUE)
        setwd(wk.dir)

        ###----- Run GSEA
        for (i in tail(colnames(z.tb), -1)){
            useGroup <- i
            outName <- i
            gsea <- tryCatch({
                    GSEA_analysis(z.file, useGroup, outName, gs.file, 1)
                }, error=function(cond) {
                    message(paste(z.use.key, gs.file, i, sep='; '))
                })
        }

        ###----- Summarize
        setwd(sum.dir)

        gsea.sum.name <- paste(z.use.key, "gsea", gs.file.base, sep="_")

        gs.all.tb <- read_csv(gs.file)
        path.order <- str_sort(unique(gs.all.tb$gs_name), numeric = TRUE)
        gs.file.ext <- basename(gs.file)
        gs.file.ext <- paste("---", gs.file.ext, sep="")

        comp.order <- sort(tail(colnames(z.tb), -1))

        gsea.files <- list.files(path = wk.dir, pattern = "*.csv", full.names = TRUE)
        gsea.file.base <- gsub(gs.file.ext, "", basename(gsea.files))
        comp.order <- intersect(comp.order, gsea.file.base)
        comp.order <- str_sort(comp.order, numeric = TRUE)
        print(comp.order)

        sum_df <- GSEA_sum(gsea.files, gsea.sum.name, 40, 20, TRUE, comp.order, TRUE, path.order)
    }
}

Parsed with column specification:
cols(
  gene_name = col_character(),
  `0` = col_double(),
  `1` = col_double(),
  `2` = col_double(),
  `3` = col_double(),
  `4` = col_double(),
  `5` = col_double(),
  `6` = col_double(),
  `7` = col_double(),
  `8` = col_double(),
  `9` = col_double()
)

Parsed with column specification:
cols(
  gene_name = col_character(),
  `0` = col_double(),
  `1` = col_double(),
  `2` = col_double(),
  `3` = col_double(),
  `4` = col_double(),
  `5` = col_double(),
  `6` = col_double(),
  `7` = col_double(),
  `8` = col_double(),
  `9` = col_double()
)

Warning message:
“Unknown columns: `gene_names`”
Parsed with column specification:
cols(
  gs_name = col_character(),
  gene_symbol = col_character()
)

preparing geneSet collections...

calculating observed enrichment scores...

calculating permutation scores...

calculating p values...

leading edge analysis...

done...

Parsed with column specification:
cols(
  gene_name = col_character(),
  `0` = col_double